<a href="https://colab.research.google.com/github/mthakurd/Mithilesh_INFO3530_Spring2020/blob/master/Assignment11-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. There is an csv file named “ch02-data.csv”. It stores a date and a numbers. Write a series of functions that can sum the numbers by date and output the results into a excel. The requirements are follows. 1.1 create a table in MySQL database which has the same structure with csv file. 1.2 read the csv file and import the data into MySQL table 1.3 using sql to sum the numbers by date 1.4 output the sum results into an excel file

In [0]:
import csv
import pymysql
import pandas as pd
import openpyxl

def read_csv(filename):
    with open(filename, 'r') as f:
        data = csv.reader(f)
        data_arr = []
        for row in data:
            data_arr.append(row)
        return data_arr


def insertData(csv_data):
    db = pymysql.connect("localhost","root","Alfa2020","asign11")
    dbcursor = db.cursor()

    csv_data = csv_data[1:]
    for row in csv_data:
        sql = "insert into prob_1(dates, numbers) values(%s,%s)"
        values = (row[0], row[1])

        dbcursor.execute(sql, values)
        db.commit()

    dbcursor.close()
    db.close()


def find_sum():
    db = pymysql.connect("localhost","root","Alfa2020","asign11")
    dbcursor = db.cursor()

    sql = "select dates, sum(numbers) as sum from prob_1 group by dates"
    dbcursor.execute(sql)
    results = dbcursor.fetchall()
    dates = []
    sum_num = []
    for row in results:
        date = row[0].strftime("%d/%m/%Y")
        dates.append(date)
        sum_num.append(row[1])
    return dates, sum_num


def write_to_excel(dates, sum_num):
    d = {'dates': dates, 'sum': sum_num}
    filepath = './output.xlsx'
    writer = pd.ExcelWriter(filepath)

    df = pd.DataFrame(d)
    df.to_excel(writer, columns=['dates', 'sum'], index=False, encoding='utf-8', sheet_name='Results')
    writer.save()


csv_data = read_csv('./assign11-1-ch02-data.csv')
insertData(csv_data)
dates, sum_num = find_sum()
write_to_excel(dates, sum_num)

2. There is an excel file named “catering_sale_all.xls”. It stores the sale situation of several kinds of food on one day. Write a function that can get the best selling food(foodname and sales) by the specific day. Use any method which you may think is easy.

solution 1

In [0]:
import pandas as pd
df = pd.read_excel("./assign11-1-catering_sale_all.xls")
result = []
food = {1: 'chicken feet', 2: 'steamed dumplings', 3: 'ribs', 4: 'full chicken', 5: 'stuffed bun', 6: 'flowering cabbage', 7: 'doufu', 8: 'chive dumplings', 9: 'pudding', 10: 'original cabbage'}
print(len(df.index))
for i in range(len(df.index)):
    max = 0
    result1 = []
    for j in range(10):
        if df.loc[i, food[j+1]] >= max:
            max = df.loc[i, food[j+1]]
            food_item = food[j+1]
    date = df.index.values[i]
    result1.append(date)
    result1.append(food_item)
    result1.append(max)
    result.append(result1)
print(result)

solution 2

In [0]:
import pymysql
import pandas as pd
import xlrd
import datetime

file="./assign11-1-catering_sale_all.xls"

def read_excel(filepath):

    wb=xlrd.open_workbook(filename=file)
    ws=wb.sheet_by_name("Sheet1")

    dataset=[]

    for r in range(ws.nrows):
        col=[]
        workbook_datemode = wb.datemode
        for c in range(ws.ncols):
            if r != 0 and c == 0:
                y, m, d, h, m, s = xlrd.xldate_as_tuple(ws.cell(r,c).value, workbook_datemode)
                date = datetime.datetime(y, 1, d)
                date = date.strftime("%Y/%m/%d")
                col.append(date)
            elif r != 0 and c != 0:
                val = ws.cell(r,c).value
                val = int(val)
                col.append(val)
            else:
                col.append(ws.cell(r, c).value)
        dataset.append(col)
    return dataset[1:]


def insertData(excel_data):
    db = pymysql.connect("localhost","root","Alfa2020","asign11")
    dbcursor = db.cursor()

    for row in excel_data:
        sql = "insert into prob_2(sale_date, chicken_feet, steamed_dumplings, ribs, full_chicken, stuffed_bun, flowering_cabbage, doufu, chive_dumplings, pudding, original_cabbage) values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        values = (row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10])

        dbcursor.execute(sql, values)
        db.commit()

    dbcursor.close()
    db.close()


def best_selling_food():
    db = pymysql.connect("localhost","root","Alfa2020","asign11")
    dbcursor = db.cursor()

    sql = "select sale_date, sum(numbers) as sum from prob_2 group by dates"
    dbcursor.execute(sql)
    results = dbcursor.fetchall()
    dates = []
    sum_num = []
    for row in results:
        date = row[0].strftime("%d/%m/%Y")
        dates.append(date)
        sum_num.append(row[1])
    return dates, sum_num

excel_data = read_excel(file)
insertData(excel_data)